In [1]:
import torch
from torch import nn
from torch.nn import functional as F 

# print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
print(device)

cuda:0


In [2]:
from GCNet.GCNet import GCNet

net = GCNet(device)
print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in net.parameters()])))

net = net.to(device)
net.train()
optimizer = torch.optim.AdamW(net.parameters(), lr=0.01)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, init_lr, num_steps + 100,
#                                           pct_start=0.01, cycle_momentum=False, anneal_strategy='linear')
    



Number of model parameters: 2791905


In [3]:
from utils.stereo_datasets import fetch_dataset, DFC2019
from tqdm import tqdm

train_loader = fetch_dataset(dataset_name = 'DFC2019', root = '/home/lab1/datasets/DFC2019_track2_grayscale_8bit',
                            batch_size = 1, resize = [512, 512], mode = 'training')

for i_batch, data_blob in enumerate(tqdm(train_loader)):
    imgl, imgr, disp, valid = [x.to(device) for x in data_blob]
    break

  0%|          | 0/4077 [00:00<?, ?it/s]


In [4]:
disp_predict = net(imgl, imgr, 0, 64)

print(disp_predict.shape)

torch.Size([1, 1, 512, 512])


In [5]:
print(disp_predict.dtype)

torch.float32


In [6]:
class SmoothL1Loss(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, disp1,  target):
        loss1 = F.smooth_l1_loss(disp1, target)
        return loss1

In [7]:
criterion = SmoothL1Loss().to(device)

optimizer.zero_grad()


In [9]:
loss = criterion(disp_predict, disp)


/tmp/ipykernel_27167/3871178227.py:7: UserWarning: Using a target size (torch.Size([1, 512, 512])) that is different to the input size (torch.Size([1, 1, 512, 512])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss1 = F.smooth_l1_loss(disp1, target)


In [ ]:
loss.backward()


In [ ]:
optimizer.step()

In [2]:
from PIL import Image
import numpy as np

filename = '/home/lab1/datasets/DFC2019_track2_grayscale_8bit/Track2-RGB-1/JAX_068_001_012_RIGHT_RGB.tif'
img = Image.open(filename)
w, h = img.size
print(w, h)
size = [512,512]
img = img.resize(size)
img = np.array(img)
img.shape

1024 1024


(512, 512, 3)

  0%|          | 0/2038 [00:01<?, ?it/s]

torch.Size([2, 3, 512, 512])
torch.Size([2, 3, 512, 512])
torch.Size([2, 512, 512])
torch.Size([2, 512, 512])
